In [38]:
#!pip install dash_bootstrap_components

In [39]:
import os

In [40]:
cwd = os.getcwd()

In [41]:
import dash
from dash import dcc
from dash import html
from dash import dash_table

from collections import OrderedDict

import plotly.graph_objects as go
from plotly.subplots import make_subplots

from dash import Dash, Input, Output, callback

from jupyter_dash import JupyterDash

import dash_bootstrap_components as dbc
COMPONENT_STYLE = "/assets/my_component.css"
external_stylesheets=[dbc.themes.BOOTSTRAP]

app = JupyterDash(__name__,requests_pathname_prefix="/dash1/",routes_pathname_prefix='/dash1/',
                  external_stylesheets=external_stylesheets,
                  meta_tags=[{'name': 'viewport', 'content': 'width=device-width, initial-scale=1'}],
                 suppress_callback_exceptions=True)
# Create server variable with Flask server object for use with gunicorn
server = app.server

In [42]:
from hello import hello_layout
#app.layout = hello_layout

In [43]:
from all_data_tables import years_table, limit_types_table
from all_data_tables import experiments_table, result_types_table, spin_dependency_table
from all_data_tables import greatest_hit_table, limits_table

from all_data_tables import years_df, limit_types_df
from all_data_tables import experiments_df, result_types_df, spin_dependency_df
from all_data_tables import greatest_hit_df, limits_df

from all_data_tables import limits_table_df

In [44]:
#app.layout = html.Div(['Hello World'])

#app.run_server(host="0.0.0.0", port=5051)

In [45]:
import pandas as pd

# Color Code

In [46]:
import plotly.graph_objects as go
import plotly.express as px
from itertools import cycle

# colors
palette = cycle(px.colors.qualitative.Bold)
#palette = cycle(['black', 'grey', 'red', 'blue'])

# SQL

In [47]:
experiments_df

,label,value
0,Theory,Theory
1,CDMS II (Soudan),CDMS II (Soudan)
2,CDMS I (SUF),CDMS I (SUF)
3,SuperCDMS,SuperCDMS
4,COSME,COSME
...,...,...
58,DRIFT,DRIFT
59,GAMBIT,GAMBIT
60,CDEX-10,CDEX-10
61,NEWS-G,NEWS-G


In [48]:
result_ids = [1,262]

multiresult_df = limits_df[limits_df['id'].isin(result_ids)]

multiresult_df.columns

Index(['id', 'spin_dependency', 'result_type', 'measurement_type', 'nomhash',
       'x_units', 'y_units', 'x_rescale', 'y_rescale', 'default_color',
       'default_style', 'data_values', 'data_label', 'file_name',
       'data_comment', 'data_reference', 'created_at', 'updated_at',
       'creator_id', 'experiment', 'rating', 'date_of_announcement', 'public',
       'official', 'date_official', 'greatest_hit', 'date_of_run_start',
       'date_of_run_end', 'year', 'rowid'],
      dtype='object')

In [49]:
#multiresult_df.to_csv(cwd+'/data.csv')

In [50]:
#csv_df = pd.read_csv(cwd+'/data.csv')
#csv_df

In [51]:
# Define scale factors [WT]
def get_scale_factor(unit):
    BARN_CM2= 1e-24
    
    if (unit == "b"):
        return BARN_CM
    elif (unit == "mb"):
        return 1e-3*BARN_CM2
    elif (unit == "ub"):
        return 1e-6*BARN_CM2
    elif (unit == "nb"):
        return 1e-9*BARN_CM2
    elif (unit == "pb"):
        return 1e-12*BARN_CM2
    elif (unit == "fb"):
        return 1e-15*BARN_CM2
    elif (unit == "ab"):
        return 1e-18*BARN_CM2
    elif (unit == "zb"):
        return 1e-21*BARN_CM2
    elif (unit == "yb"):
        return 1e-24*BARN_CM2
    else: 
        return 1

In [52]:
def datastring2dataframe(row_in):
    #try:
    #data_list = row_in['data_values'].tolist()
    #data_list = row[['data_values']].iloc[0]
    #if isinstance(data_list, pd.DataFrame):
    #    row_data = next(data_list.iterrows())[1]
    #else:
    #    row_data = data_list
    data_string = row_in[['data_values']].iloc[0]
    data_string = data_string.replace("{[", "")
    data_string = data_string.replace("]}", "")
    x = data_string.split(";")
    lol = []
    
    
    data_series = data_string.split("]")
    #print(len(data_series))
    for l in range(0,len(data_series)):
        next_colour = next(palette)
        single_set = data_series[l]
        set_list = single_set.split(";")
        for i in set_list:
            z = i.split(" ");
            new_x = z[0].replace(",[", "")
            try:
                appendthis = [row_in['id'],row_in['data_label'],l,new_x,z[1],next_colour]
            except:
                appendthis = [row_in['id'],l,0,0]
            lol.append(appendthis)
    
    #for i in x:
    #    z = i.split(" ");
    #    appendthis = [z[0],z[1]]
    #    lol.append(appendthis)
    #lol
    df_experiment = pd.DataFrame(data=lol,columns=['id','data_label','series','raw_x','raw_y','suggested_color'])

    unit = "zb" # this will probably be provided by the user via a drop-down menu
    scale_factor = float(get_scale_factor(unit))


    df_experiment['x'] = df_experiment['raw_x'].astype(str).astype(dtype = float, errors = 'ignore')

    # add scale_factor here
    df_experiment['y'] = df_experiment['raw_y'].astype(str).astype(dtype = float, errors = 'ignore')#/scale_factor
    #df_experiment['y'] =  df_experiment['y'].div(scale_factor).round(2)
    df_experiment['scale_factor'] = scale_factor
    df_experiment['scaled_y']=df_experiment[['y','scale_factor']].apply(pd.to_numeric,errors='coerce').fillna(0).eval('y/scale_factor')
    #'experiment',
    #'series'
    
    #except:
    #    data_null = [[0,0]]
    #    df_experiment = pd.DataFrame(data=data_null,columns=['raw_x','raw_y'])#

    #    df_experiment['x'] = df_experiment['raw_x'].astype(str).astype(float)
    #    df_experiment['y'] = df_experiment['raw_y'].astype(str).astype(float)
    #df_experiment.dtypes
    return df_experiment

In [53]:
multiresult_df

,id,spin_dependency,result_type,measurement_type,nomhash,x_units,y_units,x_rescale,y_rescale,default_color,...,rating,date_of_announcement,public,official,date_official,greatest_hit,date_of_run_start,date_of_run_end,year,rowid
0,1,SI,Th,Dir,b'--- \nknikolic: false\n',GeV,cm^2,,,LtR,...,0,2003-01-01,1,1,None,0,None,None,2003.0,0
244,262,SI,Th,Dir,b'--- \nknikolic: false\n',GeV,cm^2,,,DkG,...,0,2007-01-01,1,1,2009-07-16,0,None,None,2007.0,244


In [54]:
data_df = multiresult_df

lol = []
df_experiment_all=pd.DataFrame()
for index, row in data_df.iterrows():
    df_experiment_one =  datastring2dataframe(row)
    if df_experiment_all.empty:
        df_experiment_all = df_experiment_one
    else:
        df_experiment_all = pd.concat([df_experiment_all, df_experiment_one])

        
df_experiment_all        
#pd.DataFrame(data=lol,columns=['experiment','series','raw_x','raw_y','series_color'])

#unit = "zb" # this will probably be provided by the user via a drop-down menu
#scale_factor = get_scale_factor(unit)


#df_experiment['x'] = df_experiment['raw_x'].astype(str).astype(dtype = float, errors = 'ignore')

# add scale_factor here
#df_experiment['y'] = df_experiment['raw_y'].astype(str).astype(dtype = float, errors = 'ignore')/scale_factor

   

,id,data_label,series,raw_x,raw_y,suggested_color,x,y,scale_factor,scaled_y
0,1,"Baer et. al., 2003, constraints on mSUGRA, SI",0,60,4e-43,"rgb(127, 60, 141)",60.00000,4.000000e-43,1.000000e-45,400.00000
1,1,"Baer et. al., 2003, constraints on mSUGRA, SI",0,70,4e-43,"rgb(127, 60, 141)",70.00000,4.000000e-43,1.000000e-45,400.00000
2,1,"Baer et. al., 2003, constraints on mSUGRA, SI",0,160,5.5e-44,"rgb(127, 60, 141)",160.00000,5.500000e-44,1.000000e-45,55.00000
3,1,"Baer et. al., 2003, constraints on mSUGRA, SI",0,320,2.5e-44,"rgb(127, 60, 141)",320.00000,2.500000e-44,1.000000e-45,25.00000
4,1,"Baer et. al., 2003, constraints on mSUGRA, SI",0,630,3.2e-44,"rgb(127, 60, 141)",630.00000,3.200000e-44,1.000000e-45,32.00000
...,...,...,...,...,...,...,...,...,...,...
443,262,"Roszkowski et. al., 2007, CMSSM Markov Chain M...",8,124.8077,3.0205e-46,"rgb(249, 123, 114)",124.80770,3.020500e-46,1.000000e-45,0.30205
444,262,"Roszkowski et. al., 2007, CMSSM Markov Chain M...",8,130.1062,2.7127e-46,"rgb(249, 123, 114)",130.10620,2.712700e-46,1.000000e-45,0.27127
445,262,"Roszkowski et. al., 2007, CMSSM Markov Chain M...",8,45.00936,2.5253e-46,"rgb(249, 123, 114)",45.00936,2.525300e-46,1.000000e-45,0.25253
446,262,"Roszkowski et. al., 2007, CMSSM Markov Chain M...",8,142.8419,2.8124e-46,"rgb(249, 123, 114)",142.84190,2.812400e-46,1.000000e-45,0.28124


In [55]:
plotseries = df_experiment_all[['id','data_label','series','suggested_color']]
plotseries = plotseries.drop_duplicates()
plotseries['color'] = 'black'
plotseries['line'] = 'solid'
plotseries['symbol'] = 'square'
plotseries

,id,data_label,series,suggested_color,color,line,symbol
0,1,"Baer et. al., 2003, constraints on mSUGRA, SI",0,"rgb(127, 60, 141)",black,solid,square
0,262,"Roszkowski et. al., 2007, CMSSM Markov Chain M...",0,"rgb(17, 165, 121)",black,solid,square
304,262,"Roszkowski et. al., 2007, CMSSM Markov Chain M...",1,"rgb(57, 105, 172)",black,solid,square
323,262,"Roszkowski et. al., 2007, CMSSM Markov Chain M...",2,"rgb(242, 183, 1)",black,solid,square
362,262,"Roszkowski et. al., 2007, CMSSM Markov Chain M...",3,"rgb(231, 63, 116)",black,solid,square
371,262,"Roszkowski et. al., 2007, CMSSM Markov Chain M...",4,"rgb(128, 186, 90)",black,solid,square
384,262,"Roszkowski et. al., 2007, CMSSM Markov Chain M...",5,"rgb(230, 131, 16)",black,solid,square
404,262,"Roszkowski et. al., 2007, CMSSM Markov Chain M...",6,"rgb(0, 134, 149)",black,solid,square
432,262,"Roszkowski et. al., 2007, CMSSM Markov Chain M...",7,"rgb(207, 28, 144)",black,solid,square
441,262,"Roszkowski et. al., 2007, CMSSM Markov Chain M...",8,"rgb(249, 123, 114)",black,solid,square


In [56]:
import plotly.graph_objects as go
from plotly.validators.scatter.marker import SymbolValidator

raw_symbols = SymbolValidator().values
#raw_symbols

In [57]:
color_list = ['red','orange','yellow','lime green', 'green', 'blue-green', 'cyan',
              'sky blue', 'blue', 'purple', 'magenta', 'pink']

In [58]:
color_options=[{'label': i, 'value': i} for i in color_list]

In [59]:
line_styles_list = ['solid', 'dot', 'dash', 'longdash', 'dashdot', 'longdashdot']

In [60]:
line_styles_options=[{'label': i, 'value': i} for i in line_styles_list]

In [61]:
symbol_list = ['circle','square','diamond','cross','x','hexagon','pentagon','octagon','star','asterisk','hash']

In [62]:
symbol_options=[{'label': i, 'value': i} for i in symbol_list]

In [63]:
#symbol_table = dash_table.DataTable(df.to_dict('records'), [{"name": i, "id": i} for i in df.columns])

In [64]:
df = pd.DataFrame(OrderedDict([
    ('experiment', ['1', '2', '3', '4']),
    ('color', ['red', 'green', 'blue', 'yellow']),
    ('line', ['solid', 'solid', 'solid', 'solid']),
    ('symbol', ['circle','square','diamond','cross']),
]))


format_table_div = html.Div([
    dash_table.DataTable(
        id='format_table_id',
        row_deletable=True,
        # Add this line
        data=plotseries.to_dict('records'),
        columns=[
            {'id': 'id', 'name': 'id'},
            {'id': 'data_label', 'name': 'data_label'},
            {'id': 'series', 'name': 'series'},
            {'id': 'color', 'name': 'color', 'presentation': 'dropdown'},
            {'id': 'line', 'name': 'line', 'presentation': 'dropdown'},
            {'id': 'symbol', 'name': 'symbol', 'presentation': 'dropdown'},
        ],

        editable=True,
        css=[
            {"selector": ".Select-menu-outer", "rule": "display: block !important"},
            {"selector": ".dash-spreadsheet tr th", "rule": "height: 5px;"},  # set height of header
            {"selector": ".dash-spreadsheet tr td", "rule": "height: 5px;"}  # set height of body rows
            ],
        
        dropdown={
            'color': {
                'options': [
                    {'label': i, 'value': i}
                    for i in color_list
                ]
            },
            'line': {
                 'options': [
                    {'label': i, 'value': i}
                    for i in line_styles_list
                ]
            },
            'symbol': {
                 'options': [
                    {'label': i, 'value': i}
                    for i in symbol_list
                ]
            }
        },
        style_cell={'fontSize':12, 'font-family':'sans-serif', 'padding':'0px'},
        style_cell_conditional=[
            {'if': {'column_id': 'id'},
             'width': '5%'},
            {'if': {'column_id': 'data_label'},
             'width': '70%'},
            {'if': {'column_id': 'series'},
             'width': '5%'},
            {'if': {'column_id': 'color'},
             'width': '5%'},
            {'if': {'column_id': 'lime'},
             'width': '5%'},
             {'if': {'column_id': 'color'},
             'width': '5%'},
            {'if': {'column_id': 'symbol'},
             'width': '5%'}],
    ),
    html.Div(id='table-dropdown-container')
])


In [65]:
#fig = go.Figure(data=[go.Scatter(x=[1, 2, 3], y=[4, 1, 2])])

#graph = dcc.Graph(figure=fig)

#layout10 =  html.Div(format_table)


In [66]:
result_ids = [1,262]

multiresult_df = limits_df[limits_df['id'].isin(result_ids)]

plot_experiment_ids = multiresult_df['id'].unique().tolist()

plot_experiment_ids

[1, 262]

In [73]:
# Create figure
fig = go.Figure()

for pe_id in plot_experiment_ids:
    experiment_data = multiresult_df[multiresult_df['id']==pe_id]
    trace_data =  datastring2dataframe(experiment_data.iloc[0])
    series_max = trace_data['series'].max()

    line_color=color_list[1]
    line_style = line_styles_list[1]

    # Loop df columns and plot columns to the figure
    for i in range(1, series_max):
        trace2add = trace_data[trace_data['series']==i]
        fig.add_trace(go.Scatter(x=trace2add['x'], y=trace2add['scaled_y'],
                            mode='lines', # 'lines' or 'markers'
                            line=dict(width=4,dash=line_style,color=line_color),
                            #showscale=False,
                            name=i))
        fig.add_trace(go.Scatter(x=trace2add['x'], y=trace2add['scaled_y'],
                            mode='markers', # 'lines' or 'markers'
                            marker_symbol=symbol_list[1],
                                 marker=dict(
                                size=10,
                                color=line_color,#set color equal to a variable
                                #colorscale='Viridis', # one of plotly colorscales
                                showscale=False,
                            ),
                            name=i))

fig.update_xaxes(type="log")
fig.update_yaxes(type="log")

fig.update_layout(showlegend=False)

fig.update_layout(
#xaxis_title=filtered_df['x_units'].iloc[0],
#yaxis_title=filtered_df['y_units'].iloc[0],
title="Multi Series",
legend_title="Legend Title",
font=dict(
    family="Courier New, monospace",
    size=18,
    color="RebeccaPurple"
)
)

graph_out_div = html.Div(children=[dcc.Graph(figure=fig)],id='graph_out_id')

#fig.show()

In [74]:
color_dropdown = dcc.Dropdown(id='color-dropdown-id',options=color_options, value='red' )
line_dropdown = dcc.Dropdown(id='line-dropdown-id',options=line_styles_options, value='solid' )
symbol_dropdown = dcc.Dropdown(id='symbol-dropdown-id',options=symbol_options, value='diamond' )


app.layout = html.Div([
    #color_dropdown,
    #line_dropdown,
    #symbol_dropdown,
    graph_out_div,
    format_table_div,
    html.Div(id='dd-output-container')
])

@app.callback(
    [Output('graph_out_id', 'figure')],
    [Input('format_table_id', 'rows')])
    # Input('format_table_id', 'selected_row_indices')])
def update_figure(rows_in):#, selected_row_indices):
    fig1 = go.Figure()
    dff = pd.DataFrame(rows_in)
    for index, row in dff.iterrows():
        trace_data = df_experiment_all[(df_experiment_all['id']==row['id']) & (df_experiment_all['series']==row['series'])]

        line_color=color_list[1]
        line_style = line_styles_list[1]

        trace2add = trace_data
        
        fig1.add_trace(go.Scatter(x=trace2add['x'], y=trace2add['scaled_y'],
                            mode='lines', # 'lines' or 'markers'
                            line=dict(width=4,dash=line_style,color=row['color']),
                            #showscale=False,
                            name=i))
        fig1.add_trace(go.Scatter(x=trace2add['x'], y=trace2add['scaled_y'],
                            mode='markers', # 'lines' or 'markers'
                            marker_symbol=symbol_list[1],
                                 marker=dict(
                                size=10,
                                color=row['color'],#set color equal to a variable
                                #colorscale='Viridis', # one of plotly colorscales
                                showscale=False,
                            ),
                            name=i))

    graph_out = html.Div(children=[dcc.Graph(figure=fig)])
    return graph_out


#@app.callback(
#    Output('dd-output-container', 'children'),
#    Input('color-dropdown-id', 'value'),
#    #Input('line-dropdown-id', 'value'),
#    #Input('symbol-dropdown-id', 'value')
#)
#def update_output(chosen_color, chosen_line_style, chosen_symbol):
#    return f'You have selected {chosen_color}'##, chosen_line_style, chosen_symbol}'


In [75]:
#app.layout = layout10
app.run_server(host="0.0.0.0", port=5051, debug=True, use_reloader=False) 

Dash app running on http://0.0.0.0:5051/dash1/


In [70]:
#app.run_server(host='0.0.0.0', port=5051)

In [34]:
#app.run_server(debug=True,host='0.0.0.0', port=5051)

In [35]:
#app.run_server(mode='jupyterlab', port = 5052, dev_tools_ui=True, #debug=True,
#              dev_tools_hot_reload =True, threaded=True)#

In [36]:
#app.run_server(host="0.0.0.0", port=5051)

In [37]:
#app.layout = layout10
#app.run_server(host="0.0.0.0", port=5051, debug=True, use_reloader=False) 